In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
.imagesource {
    font-size: xx-small;
}
</style>
""")

from datascience import *

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')


import os
import networkx as nx

from IPython.core.display import HTML
def css_styling():
    styles = open("custom_style.css", "r").read()
    return HTML(styles)
css_styling()

## Lab 07 - Enriching network data and testing a hypothesis

Last week, we introduced a new theoretical concept: triadic closure. We learned that this idea predicts that social networks will tend to have clumps of densely interconnected groups of nodes. We explored this idea by comparing real social networks to simulated Erdos Renyi random networks.

This week, we're going to continue what we started by building up to calculating a $p$ value. We'll be looking at homophily, rather than transitive closure this time.

## Attributes of nodes

So far, we have seen that there is a lot we can learn about social networks by just studying their structure---i.e., by investigating the patterns of nodes and edges. However, since social networks are typically made of people, we know that that only studying their structure leaves a lot of potentially important information out of our analysis. People are not all the same, and so reducing people to identical nodes in a network can be too simplistic to accurately learn about the social processes that a network is a part of.

Today, we're going to start to enrich the complete network datset that we have been learning about by adding additional information about who is in the nodes. We'll continue to work with the theoretical idea of homophily, which we have discussed in a few previous classes.

### Defining homophily mathematically

If a network's nodes can be divided into discrete groups, then a quantitative metric for the amount of homophily in a network was proposed by [Newman](https://arxiv.org/pdf/cond-mat/0205405.pdf). We'll work with this metric today.

* suppose that the nodes can all be divided into discrete groups
* let $a_i$ be the fraction of ends of edges ('stubs') in a network that are connected to nodes in group $i$
* let $e_{ij}$ be the fraction of edges in a network ('stubs') that connect nodes in group $i$ to nodes in group $j$


Newman's *assortativity coefficient* is:

$$r = \frac{\sum_i e_{ii} - \sum_i a_i^2}{1 - \sum_i a_i^2}$$

* when $r=0$, then there is no assortativity
* when $r>0$, then there is assortativity -- i.e., members of the same group tend to be more connected to each other than to other groups. At the extreme, when $r=1$, all network connections are within groups
* when $r<1$, then there is disassortativity -- i.e., members of the same group are less likely to be connected to each other than to other groups

For two groups, we can write the assortativity coefficient as

$$r = \frac{e_{11} + e_{22} - a_1^2 - a_2^2}{1 - a_1^2 - a_2^2}$$

### Group example

<img src='example_network_groups3.png'>

**Group Discussion Question** Does this network look homophilous?

**Group Discussion Question** Calculate the assortatitivty coefficient in the example network:

|             |  degree |   group |
|   :----:    |  :---:  |   :---: |
|   node 1    |    1    |    blue |
|   node 2    |    2    |    red  |
|   node 3    |    5    |    blue |
|   node 4    |    1    |    red  |
|   node 5    |    2    |    blue |
|   node 6    |    1    |    red  |
|   node 7    |    2    |    red  |
|   node 8    |    2    |    blue |


*Stub counts*

|             | blue | red   |
|   :----:    | :---:| :---: |
|   blue      |  4   |  6    |
|   red       |  6   |  0    |

*Stub totals*

|             |  total degree | $a_i$ |
|   :----:    |  :---:        | :---: |
|   blue      |    10         | 11/16 |
|   red       |    6          | 5/16  |

*$e_{ij}$ matrix*

|             |  blue | red   |
|   :----:    |  :---:| :---: |
|   blue      |  4/16 |  6/16 |
|   red       |  6/16 |  0    |

*assortativity coefficient*
$r$ = [((4/16) + 0) - (10/16)^2 - (6/16)^2] / [1 - (10/16)^2 - (6/16)^2] = 0.2

In [ ]:
(((4/16) + 0) - (10/16)**2 - (6/16)**2) / (1 - (10/16)**2 - (6/16)**2)

### [Randomize partners]

**Question** What is your partner's name?

[answer here]

**Question** What is your partner's favorite food?

[answer here]

OK, back to work. Now we'll use the `networkx` package to check the results of the calculation we performed above. This will  give us a chance to learn about how the package handles node attributes.

**Question** Create a `networkx` graph called `test` that represents the example network above.<BR>
*[Hint: you can enter this network as an edgelist]*

In [ ]:
test = nx.Graph([(1,3), (2,3), (2,5), (3,4), (3,7), (3,8), (5,6), (7,8)])

The pattern below shows us how to tell `networkx` that there is a node attribute called 'color', and it shows us how to explicitly add the color of each node:

In [ ]:
nx.set_node_attributes(test,
                       'color',
                       {1 : 'blue',
                        2 : 'red',
                        3 : 'blue',
                        4 : 'red',
                        5 : 'blue',
                        6 : 'red',
                        7 : 'red',
                        8 : 'blue'})

See the help page for [`set_node_attributes`](https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.classes.function.set_node_attributes.html) for more information on working with attributes.

You can check the matrix of $e_{ij}$ values by using the function `nx.attribute_mixing_matrix`:

In [ ]:
nx.attribute_mixing_matrix(test, 'color')

Note that, by default, the order of the columns of the mixing matrix is not specified. If you want to be able to interpret the columns, you can pass in a dictionary that maps the different attribtues to column numbers like we do below:

In [ ]:
nx.attribute_mixing_matrix(test, 'color', mapping = {'red' : 1, 'blue' : 0})

We can calculate the value of the assortativity coefficient using the `nx.attribute_assortativity_coefficient` function:

In [ ]:
nx.attribute_assortativity_coefficient(test, 'color')

**Question** With your partner, look at the next example network. Does it look more or less homophilous by color than the one above?

<img src='example_network_groups2.png'>

**Question** With your partner, calculate the assortativity coefficient for this network by filling in the missing quantities below.

**Group Discussion Question** Calculate the assortatitivty coefficient in the example network:

|             |  degree |   group |
|   :----:    |  :---:  |   :---: |
|   node 1    |    1    |    blue |
|   node 2    |    2    |    red  |
|   node 3    |    5    |    blue |
|   node 4    |    1    |    blue  |
|   node 5    |    2    |    red |
|   node 6    |    1    |    red  |
|   node 7    |    2    |    blue  |
|   node 8    |    2    |    blue |


*Stub counts*

|             | blue | red   |
|   :----:    | :---:| :---: |
|   blue      |  ?  |  ?    |
|   red       |  ?   |  ?    |

*Stub totals*

|             |  total degree | $a_i$ |
|   :----:    |  :---:        | :---: |
|   blue      |    ?         | ? |
|   red       |    ?          | ?  |

*$e_{ij}$ matrix*

|             |  blue | red   |
|   :----:    |  :---:| :---: |
|   blue      |  ? |  ? |
|   red       |  ? |  ?    |

*assortativity coefficient*
$r$ = ?

**Question** Now check your answer using the `networkx` package.

In [ ]:
test2 = nx.Graph([(1,3), (2,3), (2,5), (3,4), (3,7), (3,8), (5,6), (7,8)])
nx.set_node_attributes(test2,
                       'color',
                       {1 : 'blue',
                        2 : 'red',
                        3 : 'blue',
                        4 : 'blue',
                        5 : 'red',
                        6 : 'red',
                        7 : 'blue',
                        8 : 'blue'})

In [ ]:
nx.attribute_mixing_matrix(...)

In [ ]:
nx.attribute_assortativity_coefficient(...)

### Reading Add Health attributes

Recall that the Add Health dataset we have been working with stores information about network connections in an edge list.  The Add Health dataset also has information about the nodes; it has their grade, their gender, and their race/ethnicity. The code below will read all of this information in for you.

Note that there were problems with two of the Add Health networks' node data, so we'll leave those two out. Our set of Add Health networks is thus 82 nodes instead of 84 for today.

In [ ]:
def read_add_health_edges(network_id):
    """
    network_id : integer from 1 to 84
    
    read in the Add Health network corresponding to the given id number and
    return it as an undirected networkx object
    """

    # this file was downloaded from
    # http://moreno.ss.uci.edu/data.html#adhealth
    edge_file = os.path.join("data", "comm" + str(network_id) + ".dat")
    with open(edge_file, 'r') as f:
        edge_lines = f.readlines()
        
    network = nx.parse_edgelist(edge_lines, nodetype=int, data=[('activity_level', float)])
    
    # note that we call the to_undirected method to ensure we get an undirected network
    return(network.to_undirected())

def read_add_health_attributes_oneperrow(network_id, net):
    """
    Read in an Add Health attributes file that has one attribute per row
    """
    att_file = os.path.join("data", "comm" + str(network_id) + "_att.dat")
    with open(att_file, 'r') as f:
        att_lines = f.readlines()
    
    # the first 8 lines are meta-info and not actual data
    att_lines = att_lines[8:]
    
    node_races = {}
    node_grades = {}
    node_sexes = {}

    for cur_id in net.nodes():
        
        print("starting node ", cur_id)
        
        # the attributes are stored one per line for each node in sequence (race / sex / grade)
        # so line 0 is node 1's race, line 2 is node 1's sex, line 3 is node 1's grade, line 4 is node 2's race, etc
        start_idx = (cur_id-1) * 3
        this_race = str.split(g_att[start_idx])[2]
        this_sex = str.split(g_att[start_idx+1])[2]
        this_grade = str.split(g_att[start_idx+2])[2]
    
        node_races[cur_id] = this_race
        node_grades[cur_id] = this_grade
        node_sexes[cur_id] = this_sex
    
    nx.set_node_attributes(net, 'race', node_races)
    nx.set_node_attributes(net, 'grade', node_grades)
    nx.set_node_attributes(net, 'sex', node_sexes)
    
    return(net)

def read_add_health_attributes(network_id, net):
    """
    Read in an Add Health attributes file that has one row per node
    """
    
    # open up the attributes datafile
    att_file = os.path.join("data", "comm" + str(network_id) + "_att.dat")
    with open(att_file, 'r') as f:
        att_lines = f.readlines()
        
    # the first several lines are meta-info and not actual data;
    # the data start once we see "DATA:\n"
    header_start = att_lines.index("COLUMN LABELS:\n") + 1
    header_end = att_lines.index("DATA:\n")
    data_start = header_end + 1
    
    # build up a list that maps column index to column name
    col_defs = []
    # build up a dict that has the data for each variable
    col_data = {}
    
    for colindex, lineidx in enumerate(range(header_start, header_end)):
        # strip off the newline and the starting/ending quotes of the column name
        this_name = (str.strip(att_lines[lineidx])[1:-1]).lower()
        col_defs.append(this_name)
        # initialize the data for this column to empty dict
        col_data[this_name] = {}  
    
    att_lines = att_lines[data_start:]
    
    # for each row (corresponding to one node's data)
    # split the columns up and stick them into the appropriate
    # dict, with node id as key and attribute value as value
    for cur_id in net.nodes():
        #print('starting node ', cur_id)
        these_data = str.split(att_lines[cur_id - 1])
        
        for colidx, dat in enumerate(these_data):
            col_data[col_defs[colidx]][cur_id] = dat

    # take the data and assign it to the nodes in the graph object
    for var in col_defs:
        nx.set_node_attributes(net, var, col_data[var])
    
    return(net)

def read_add_health_network(network_id):
    
    this_net = read_add_health_edges(network_id)
    #this_net = read_add_health_attributes(network_id, this_net)
    this_net = read_add_health_attributes(network_id, this_net)
    
    return(this_net)

Having loaded those functions, we can use the `read_add_health_network` function to load the datsets into memory:

In [ ]:
# EXCEPTIONS: networks 1 and 48 have formatting problems, so we'll omit them today
add_health_ids = [x for x in range(2, 85) if x != 48]
add_health_networks = [read_add_health_network(x) for x in add_health_ids]

### Exploring network attributes

We'll start by looking at the fraction of students in one school that is male.

Looking at the [dataset information](http://moreno.ss.uci.edu/data.html#adhealth), you can see that the Add Health sex variable has the values 1=male, 2=female, and 0=unreported.

**Question** Use the `get_node_attributes` function to grab the sexes of the students in the first school by filling in the code below.

In [ ]:
net = add_health_networks[0]
sexes = ...
sexes

It turns out that the `get_node_attributes` function returns a dictionary, but we will find it easier to work with `numpy` arrays. To convert the dictionary values into an array, use the following code:

In [ ]:
sexes_array = np.array(list(sexes.values()))

**Question** Calculate the fraction of students in the first school that is male.

In [ ]:
...

**Question** Now write a loop that calculates the fraction of students that is male in each of the Add Health schools. Store your results in an array called `add_health_frac_male`.

In [ ]:
add_health_frac_male = make_array()

...
...
...

add_health_frac_male

**Question** Make a histogram that shows the distribution of the fraction male across the schools in the sample.

In [ ]:
...

**Question** Following the pattern you used to calculate the fraction male in each community, write another loop that calculates the assortativity coefficient for sex in each community. Store your results in an array called `add_health_r`.

In [ ]:
add_health_r = make_array()

...
...
...
    
add_health_r

**Question** Plot a histogram of the assortativity coefficients across the Add Health networks.

In [ ]:
...

### Generating random networks with random sexes

Now we're in a similar situation to last week: it is somewhat difficult to interpret the absolute numbers that we obtained in the histogram above. A different approach would be to set up a *null model* and to compare what we see in the real world to what we would see if the null model were true.

For our null model, we'll use the ER random network with one twist: in addition to randomly drawing edges between the nodes, we're also going to assign each node to a random sex. Therefore, our network will have three parameters: the number of nodes, the average degree, and the fraction of nodes that is male.

We'll start by bringing in some of the code we used to generate random networks in the last lab:

In [ ]:
def er_by_degree(n, avg_degree):
    return(nx.erdos_renyi_graph(n=n, p=avg_degree / (n-1)))

def rand_er_network(network):
    """
    Return a random network generated from the configuration model using
    the degree sequence of the network passed in
    """
    network_n = network.number_of_nodes()
    network_dbar = network.number_of_edges() / network_n
    return(er_by_degree(network_n, network_dbar))

The two functions above enable us to start from a real world network and to generate a random ER network that has the same average degree and the same number of nodes.

This next function below takes a network and randomly assigns a given fraction of the nodes to be male, with the remainder set to be female:

In [ ]:
def add_random_sex(network, frac_male):
    """
    Given a network, randomly give each node a gender,
    with frac_male ending up male (we'll call this sex so that it
    is the same as the Add Health networks)
    """
    all_nodes = network.nodes()
    num_male = int(np.floor(len(all_nodes) * frac_male))
        
    genders = (['male'] * num_male) + (['female'] * (len(all_nodes) - num_male))
    genders_dict = dict(zip(all_nodes, genders))
    
    nx.set_node_attributes(network, 'sex', genders_dict)

**Question** It will be convenient to have a single function that generates a random network with nodes assigned to a specific sex, all based on an existing network. You can do this by combining the three functions above. Fill in the function definition below to accomplish this.

In [ ]:
def rand_er_network_with_sex(network):
    
    # compute fraction male
    ...
    
    # generate matching ER random network
    ...
    
    # set the right fraction of nodes to be male
    ...
    
    # return the result
    return(...)

**Question** Write a loop to generate 100 random networks with random sex. Have the random networks match the first Add Health network in terms of number of nodes, average degree, and fraction male. For each random network, calculate the assortativity coefficient and store it in an array called `er_network_r`.

In [ ]:
first_add_health = add_health_networks[0]

er_network_r = make_array()

...
...
...

**Question** Make a histogram of the `er_network_r` values. What value is the assortativity coefficient centered on?

In [ ]:
...

**Question** Now calculate the assortativity coefficient for the Add Health network that you based the random networks on. Call this `observed_r`.

In [ ]:
observed_r = ...
observed_r

**Question** We're going to use these random networks as the null model for a hypothesis test. In words, describe what our null model says about the world.<BR>
*[Hint: you can look at the [lecture slides](http://data8.org/fa16/lectures/lec19.pdf) if you don't remember what the null model is]*.

[answer here]

**Question** Now, in words, describe what the alternative model says about the world.

In [ ]:
[answer here]

**Question** Now calculate an empirical $p$ value based on your null and alternative hypotheses.

In [ ]:
emp_p_value = ...
emp_p_value

**Question** What do you conclude about homophily by sex in this network?

[Answer here]

## Submit the lab

You're almost done! Now please create a pdf version of your completed lab by **either**:

* printing your notebook to a pdf file
* going to the Jupyter 'File' menu, choosing 'Download as' and then 'PDF via LaTeX (.pdf)'. 

Please save the resulting .pdf on your computer and then **submit the .pdf on bcourses**.

**The lab must be submitted by the end of the day on Monday, Oct. 24. Late labs will not be accepted.**